## Keras model

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.model_selection import RandomizedSearchCV

from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import advanced_activations
from keras.optimizers import Adam
import keras


# data cleaning to be removed

tlc_db = pd.read_csv("TelcoChurn.csv")
tlc_db.head()
tlc_db['TotalCharges'] = tlc_db["TotalCharges"].replace(" ",0)
tlc_db["TotalCharges"] = tlc_db["TotalCharges"].astype(float)
tlc_db[['Partner','Dependents','PhoneService','PaperlessBilling','Churn']] = tlc_db[['Partner','Dependents','PhoneService','PaperlessBilling','Churn']].replace({'No': 0, 'Yes': 1})
tlc_db['gender'] = tlc_db['gender'].replace({'Male': 0, 'Female': 1})
tlc_db1 = tlc_db.copy()
columns1 = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
            'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']
for colnames in columns1:
    mlt = pd.get_dummies(tlc_db1[colnames])
    list1 = list(mlt.columns.values)
    listaname = []
    for i in list1:
        string = "" 
        for y in i.split():
            string += y.capitalize()
        listaname.append(colnames+'_'+string)
    mlt.set_axis(listaname, axis='columns', inplace=True)
    tlc_db1.drop(colnames, axis = 1, inplace=True)
    tlc_db1 = pd.concat([tlc_db1, mlt], axis = 1)
tlc_db1.drop('customerID',1,inplace=True)


sss = StratifiedShuffleSplit(test_size=0.25, n_splits=1, random_state=1234)
for train_index, test_index in sss.split(tlc_db1, tlc_db1['Churn']):
    print(len(train_index), len(test_index))
    print(tlc_db1.loc[train_index, 'Churn'].mean(), tlc_db1.loc[test_index, 'Churn'].mean())
    train = tlc_db1.loc[train_index]
    test = tlc_db1.loc[test_index]

Y = tlc_db1["Churn"]
X = tlc_db1.drop(columns=['Churn'])

x_train = train.drop(columns=['Churn'])
y_train = train["Churn"]
x_test = test.drop(columns=['Churn'])
y_test = test["Churn"]

### Model definition

In [12]:
def create_model(n_hidden_1 = 30, n_hidden_2 = 10, activation_fun = 'relu', lr = 0.01):
    print("lr " + str(lr) + "| n_hidden_1 " + str(n_hidden_1) + "| n_hidden_2 " + str(n_hidden_2) + " | activation_fun " + str(activation_fun)) 
    model = Sequential()
    model.add(Dense(num_input, input_dim=num_input,activation=activation_fun))
    model.add(Dense(n_hidden_1,activation=activation_fun))
    model.add(Dense(10,activation=activation_fun))
    model.add(Dense(1,activation=activation_fun))
    model.add(Dense(num_classes,activation='sigmoid'))
    adam = Adam(lr=lr)
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=metrics)
    return model

model = KerasRegressor(build_fn=create_model, batch_size=batch_size, epochs=10, verbose=1)

### Hyperparameters optimization by RandomizedSearchCV

In [ ]:
# Parameters
metrics = ['accuracy']
batch_size = 128
lr = [0.001, 0.003, 0.005, 0.007, 0.01]
epochs = [10, 20, 30, 40]
num_input = x_train.shape[1]
n_hidden_1 = [5, 10, 20, 50, 100] 
n_hidden_2 = [5, 10, 30, 40]
num_classes = 1
param_dist = dict(n_hidden_1=n_hidden_1, n_hidden_2=n_hidden_2, epochs=epochs, lr=lr)
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, n_jobs=-1, cv=5)
random_search.fit(X, Y)

# Show the results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Best hyperparameters
Best parameters found by random search are {'n_hidden_2': 30, 'n_hidden_1': 20, 'lr': 0.005, 'epochs': 20}

In [ ]:
model = create_model(n_hidden_1=20, n_hidden_2=30, lr= 0.005)
model.fit(x_train, y_train, batch_size=128, epochs=20)
score = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
score